In [1]:
import pyspark
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkFiles
import re
import random


path = "../data/covid_news_truncated_2.json"

conf = SparkConf()
conf.getAll()

sc = SparkContext(appName="lsh")
    
spark = SparkSession(sc)
sc.setLogLevel("ERROR")

textfile = sc.textFile(path)

#sc.addFile(path)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/22 22:50:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Large prime value to use in hashing
LARGE_PRIME = 75874811

# number of hash functions
K_VALUE = 100
R_VALUE = 5
B_VALUE = 20

# generate random array to use in hashing
RANDOM_ARRAY = [(random.randint(1,2**31 - 1), random.randint(1,2**31 - 1)) for i in range(0,K_VALUE)]


# gets document and returns k-shingles
def shingles(document, k=9):

    #initial set
    shingles_set = set()

    # getting rid of punctuation, etc
    document[1] = re.sub(r'[^\w\s]', '', document[1].lower())

    # split to chars
    chars_list = re.split('', document[1].lower())

    # create shingles with length k
    for i in range(len(chars_list) - k):
        chars = chars_list[i:i + k]
        shingle = ''.join(chars)
        shingles_set.add(shingle)

    # sort shingles
    shingles_set = sorted(shingles_set)

    # returns: doc_id, set_shingles
    return document[0], shingles_set


# hash a value
def hash_function(x, a, b, len_x): 
    return (((a * hash(x) + b) % LARGE_PRIME ) % len_x)

# get signature matrix from min hashing
def min_hash(document):
    
    # shingles
    x = document[1]

    # initial matrix
    signature_matrix = []

    # iterate through the defined number of hash functions (hi)
    for i in range(0, K_VALUE):

        # start value is infinite
        minhash = float('inf')

        # get random integers
        a,b = RANDOM_ARRAY[i]

        # for each shingle
        for value in x:
            # hash shingle
            h = hash_function(value,a,b, len(x))
            # if lower, replace with current value
            if h < minhash:
                minhash = h

        # append the lowest number
        signature_matrix.append(minhash)

    #print(signature_matrix)
    
    # returns: doc_id, signature matrix
    return document[0],signature_matrix


# ---------------------------------------------------------------------- #

# gets a band and hashes it to a bucket
def hash_lsh(band): 

    # intial array
    h1_array = []

    # for each row within the band
    for value in band:
        #print(value)
        #print((value * len(band)) % LARGE_PRIME)
        #print((421*value + 16) % 1013)
        #print("------")
        # hash
        h1_array.append((value * len(band)) % LARGE_PRIME)

    #returns: min value
    return min(h1_array)

# gets signature and returns bucket values for each band
def get_bucket_values(signature):


    # list of bucket values
    bucket_values = []

    # for the entire signature, iterate over each band with r rows and hash it
    for idx in range(0, B_VALUE):

        # if there is no more bands to hash
        if idx * R_VALUE > len(signature): 
            break 

        # get end of band
        max_id = min(idx * R_VALUE + R_VALUE, len(signature))

        # hash the band to a bucket
        bucket = hash_lsh(signature[idx * R_VALUE : max_id])

        # append bucket value
        bucket_values.append(bucket)

    # returns: doc_id, bucket values
    return bucket_values

# given the signatures, returns candidate pairs
def lsh_algorithm(signatures_matrix):

    # dict with buckets for each document
    k_buckets = {}

    # initial candidates list
    candidates = []

    # iterate over the signatures
    for doc in signatures_matrix:

        # get the bucket values for each signature
        bucket = get_bucket_values(signatures_matrix[doc])
        #print(bucket)

        # iterate over the other signatures bucket values
        for b_doc in k_buckets:

            # iterate over the bucket values
            for i in range(len(bucket)):

                # if at least 1 bucket value is the same, then at least 1 band hashes to the same bucket -> candidate 
                if k_buckets[b_doc][i] == bucket[i]:

                    # because it is candidate, compare with jaccard similarity and append to candidates list
                    similar_pair = jaccard_similarity(signatures_matrix[doc], signatures_matrix[b_doc])
                    candidates.append((doc, b_doc, similar_pair))

                    # because we only need 1 hash value in the same bucket, no need to continue
                    break

        # add the bucket values for each signature
        k_buckets[doc] = bucket

    # returns: candidates
    return candidates

# calculate jaccard similarity
def jaccard_similarity(sig_matrix_1, sig_matrix_2):
    # get intersection of the 2 matrices
    intersection = len([sig_matrix_1[i] for i in range(0, len(sig_matrix_1)) if (sig_matrix_1[i] == sig_matrix_2[i])])
    #intersection_2 = len([sig_matrix_1[i] for i in range(0, len(sig_matrix_1)) if (sig_matrix_1[i] == sig_matrix_2[i]) and (sig_matrix_1[i] != 0)])
    # get union of the 2 matrices
    #union_2 = len([sig_matrix_1[i] for i in range(0, len(sig_matrix_1)) if (sig_matrix_1[i] != 0) or (sig_matrix_2[i] != 0)])
    union = (len(sig_matrix_1) + len(sig_matrix_2)) - intersection

    # calculate jaccard similarity
    jaccard_sim = intersection / union
    #jaccard_sim_2 = intersection_2 / union_2

    return jaccard_sim


final = textfile.map(lambda line: eval(line)) \
                .map(lambda dict: [dict["tweet_id"], dict["text"]]) \
                .map(shingles) \
                .map(min_hash)

#print("shingles and minhash done")

signatures_matrix = { doc: sig_matrix for doc, sig_matrix in final.collect() }
#
#print("collect done")
#
#print(signatures_matrix)

#y = sc.parallelize(final)
#x = final.collect()
#print(x)

candidates = lsh_algorithm(signatures_matrix)
#
#print("similar pairs found")
#
sorted_candidates = sc.parallelize(candidates).sortBy(lambda pair: - pair[2])
#
#print("final pairs done")
#
final_results = sorted_candidates.collect()
#
for x in final_results[-10:]:
    print(x)


('1346071551842594817', '1346067714662670336', 0.28205128205128205)
('1346071551842594817', '1346023661002752000', 0.25)
('1346071551842594817', '1346023652303757312', 0.3157894736842105)
('1346071551842594817', '1346023643663495168', 0.35135135135135137)
('1346071551842594817', '1346026165308370944', 0.3422818791946309)
('1346071551842594817', '1346027458827608065', 0.2903225806451613)
('1346071551842594817', '1345941019414720512', 0.27388535031847133)
('1346071551842594817', '1346074073105887233', 0.26582278481012656)
('1346071551842594817', '1346074071608532994', 0.30718954248366015)
('1346071551842594817', '1346073787918331905', 0.35135135135135137)
